In [9]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import nltk


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
desc = pd.read_csv('/content/drive/MyDrive/desc.csv')

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Books/books.csv')

In [4]:
df['desc']=desc['desc']

In [5]:
df.head()

,Unnamed: 0,r_index,book_id,authors,original_title,image_url,average_rating,ratings_count,genre,desc
0,0,1,2767052,Suzanne Collins,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,4.34,4780653,"fiction, fantasy","In a future North America, where the rulers of..."
1,1,2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,https://images.gr-assets.com/books/1474154022l...,4.44,4602479,"fantasy, fiction",Rescued from the outrageous neglect of his aun...
2,2,3,41865,Stephenie Meyer,Twilight,https://images.gr-assets.com/books/1361039443l...,3.57,3866839,"fantasy, fiction",With 160 million copies of the Twilight Saga s...
3,3,4,2657,Harper Lee,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680l...,4.25,3198671,"classics, fiction",The unforgettable novel of a childhood in a sl...
4,4,5,4671,F. Scott Fitzgerald,The Great Gatsby,https://images.gr-assets.com/books/1490528560l...,3.89,2683664,"classics, fiction",A young man newly rich tries to recapture the ...


In [6]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)
# Function for converting into lower case
def make_lower_case(text):
    return text.lower()
# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text
# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text
#Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [11]:
df['cleaned_desc'] = df['desc'].apply(_removeNonAscii)
df['cleaned_desc'] = df.cleaned_desc.apply(func = make_lower_case)
df['cleaned_desc'] = df.cleaned_desc.apply(func = remove_stop_words)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_punctuation)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_html)


In [12]:
df['cleaned_desc'].head()

0    future north america rulers panem maintain con...
1    rescued outrageous neglect aunt uncle young bo...
2    160 million copies twilight saga sold worldwid...
3    unforgettable novel childhood sleepy southern ...
4    young man newly rich tries recapture past win ...
Name: cleaned_desc, dtype: object

In [18]:
def create_soup(x):
    return x['original_title'] + ' ' + x['cleaned_desc'] + ' ' + x['average_rating']

In [15]:
features = ['authors','original_title','cleaned_desc','average_rating']
df_filledbooks = df[features]
df_filledbooks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9274 entries, 0 to 9273
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   authors         9274 non-null   object 
 1   original_title  9274 non-null   object 
 2   cleaned_desc    9274 non-null   object 
 3   average_rating  9274 non-null   float64
dtypes: float64(1), object(3)
memory usage: 289.9+ KB


In [16]:
df_filledbooks = df_filledbooks.astype(str)
df_filledbooks.dtypes

authors           object
original_title    object
cleaned_desc      object
average_rating    object
dtype: object

In [19]:
df_filledbooks['soup'] = df_filledbooks.apply(create_soup, axis = 1)

In [22]:
df_filledbooks['soup']

0       The Hunger Games future north america rulers p...
1       Harry Potter and the Philosopher's Stone rescu...
2       Twilight 160 million copies twilight saga sold...
3       To Kill a Mockingbird unforgettable novel chil...
4       The Great Gatsby young man newly rich tries re...
                              ...                        
9269    Bayou Moon parents vanish cerise mar becomes t...
9270    Means of Ascent  traces johnson s life texas c...
9271    The Mauritius Command napoleonic wars british ...
9272    Cinderella Ate My Daughter: Dispatches from th...
9273    The First World War history great war chronicl...
Name: soup, Length: 9274, dtype: object

In [25]:
def recommend_new_books(title,df_filledbooks=df_filledbooks):
    
    df_filledbooks = df_filledbooks.reset_index()
    indices = pd.Series(data = df_filledbooks.index, index = df_filledbooks['original_title'])
    tfidf = TfidfVectorizer(stop_words = 'english')
    # m by n matrix
    tf_matrix = tfidf.fit_transform(df_filledbooks['soup'])

    cosine_sim2 = cosine_similarity(tf_matrix, tf_matrix)
    
    
    
    
    #title = title.replace(' ','').lower()
#   Get indice of given book
    idx = indices[title]
    
    # Get this books sim with all other books, enum to keep track of book index
    sim_scores = list(enumerate(cosine_sim2[idx]))
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Except the book itself
    sim_scores = sim_scores[1:11]
    
    movie_indices  = [i[0] for i in sim_scores]
    
    return list(df['original_title'].iloc[movie_indices])

In [26]:
recommend_new_books("Twilight")

['Twilight: The Graphic Novel,',
 'The Twilight Collection (Twilight, #1-3)',
 'Midnight Sun (Partial Draft)',
 'Twilight and History',
 'Eclipse',
 'New Moon (Twilight, #2)',
 'The Twilight Saga: The Official Guide',
 'The Twilight Saga',
 'Breaking Dawn',
 'Twilight: The Graphic Novel']

In [27]:
recommend_new_books("Harry Potter and the Philosopher's Stone")

['Harry Potter Collection (Harry Potter, #1-6)',
 'Complete Harry Potter Boxed Set',
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter Boxed Set Books 1-4',
 'Harry Potter: Film Wizardry',
 'Harry, A History: The True Story of a Boy Wizard, His Fans, and Life Inside the Harry Potter Phenomenon',
 'Harry Potter and the Half-Blood Prince',
 'Harry Potter and Philosophy: If Aristotle Ran Hogwarts',
 'Hogwarts: An Incomplete and Unreliable Guide',
 'Harry Potter and the Chamber of Secrets']

In [28]:
recommend_new_books("The Hunger Games")

['Mockingjay',
 'The Hunger Games Box Set',
 'Catching Fire',
 'Hunger ',
 'Forbidden',
 'The Long Walk',
 'Private Games ',
 "Trickster's Queen",
 'Head Games ',
 'How to Talk to a Widower']